# GGUF 変換 & 量子化ノートブック (llama.cpp)

このノートブックは **Hugging Face のモデルを GGUF に変換**し、さらに **量子化 (例: Q5_K_M)** を行います。
次の前提で動作します:

- `llama.cpp` が `/opt/llama.cpp` にあり、ビルド済みバイナリが `/opt/llama.cpp/build/bin` にあること
- Python 依存 (transformers / tokenizers / sentencepiece / safetensors / huggingface_hub / numpy) が導入済み
- 作業ディレクトリ `/workspace`、出力 `/workspace/out`、HF キャッシュ `/workspace/hf/.cache`
- JupyterLab 上で実行（このノートブックは `/workspace/notebooks` に配置想定）

ヒント: 事前に環境変数 `HF_HOME=/workspace/hf/.cache` を設定しておくとキャッシュが再利用でき効率的です。

# 0. GPUチェック（最初に実行してください）
- このセルは **GPU が正しく見えているか** を素早く確認します。
- `nvidia-smi` が利用可能なら **GPU名 / メモリ / Compute Capability** を表で表示します。
- 利用不可の場合は `/dev/nvidia*` の存在確認にフォールバックします。
- ついでに `nvcc --version` と `` も表示します。


In [ ]:
# 0. GPUチェック（最初に実行）
import os, subprocess, shutil, pandas as pd
from IPython.display import display, Markdown

def sh(cmd: str) -> str:
    try:
        out = subprocess.check_output(["bash","-lc", cmd], text=True, stderr=subprocess.STDOUT)
        return out
    except subprocess.CalledProcessError as e:
        return e.output or ""


# nvidia-smi で表を作る
q = "index,name,driver_version,memory.total,memory.free,compute_cap"
out = sh(f"nvidia-smi --query-gpu={q} --format=csv,noheader,nounits 2>/dev/null || true").strip()
rows = []
if out:
    for line in out.splitlines():
        parts = [p.strip() for p in line.split(",")]
        if len(parts) >= 6:
            rows.append({
                "GPU": parts[0],
                "Name": parts[1],
                "Driver": parts[2],
                "MemTotal(MiB)": parts[3],
                "MemFree(MiB)": parts[4],
                "ComputeCap": parts[5],
            })
if rows:
    import pandas as pd
    df = pd.DataFrame(rows)
    display(Markdown("**Detected GPU(s)**"))
    display(df)
    print()
else:
    # フォールバック：デバイスファイルの存在確認
    print("`nvidia-smi` が見つからない/利用できません。フォールバック情報:")
    try:
        out2 = sh("ls -l /dev/nvidia* 2>/dev/null || true")
        print(out2 or "NVIDIA device files not present.")
    except Exception as e:
        print("NVIDIA device files not present.")

# CUDA / nvcc
nvcc_line = sh("nvcc --version 2>/dev/null | tail -n1 || echo 'nvcc not found'").strip()
print("nvcc:", nvcc_line)

# llama.cpp のバイナリ存在チェック（簡易）
BIN = "/opt/llama.cpp/build/bin/llama-cli"
if os.path.exists(BIN):
    print("llama-cli:", BIN, "(found)")
else:
    print("llama-cli: not found at", BIN)

## 1. バージョン確認 / 依存チェック

In [ ]:
import json
mods = ["transformers","tokenizers","sentencepiece","safetensors","huggingface_hub","numpy"]
info = {}
for m in mods:
    try:
        mod = __import__(m)
        info[m] = getattr(mod, "__version__", "(no __version__)")
    except Exception as e:
        info[m] = f"IMPORT FAILED: {e}"
print(json.dumps(info, indent=2, ensure_ascii=False))

## 2. 変換パラメータの設定

- `MODEL_REPO` : Hugging Face のモデルID あるいはローカルパス
- `OUT_DIR` : 出力先ディレクトリ
- `OUT_BASENAME` : 出力ファイル名のベース
- `OUTTYPE` : f16 / f32 など

例として **sarashina** を既定にしています。

In [ ]:
import os
MODEL_REPO = "sbintuitions/sarashina2.2-3b-instruct-v0.1"  # 例
OUT_DIR = "/workspace/out"
OUT_BASENAME = "sarashina3b"
OUTTYPE = "f16"  # f16 / f32 など
QUANT = "Q5_K_M"  # 推奨量子化（バランス良）

os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs("/workspace/hf", exist_ok=True)
os.environ.setdefault("HF_HOME", "/workspace/hf/.cache")
print("HF_HOME=", os.environ["HF_HOME"]) 
print("OUT_DIR=", OUT_DIR)
print("MODEL_REPO=", MODEL_REPO)


## 3. GGUF (F16) へ変換

In [ ]:
import os, sys, re, time, threading, queue, subprocess
from pathlib import Path
from ipywidgets import FloatProgress, HBox, HTML
from IPython.display import display, clear_output

conv_script  = "/opt/llama.cpp/convert_hf_to_gguf.py"
outfile_f16  = f"{OUT_DIR}/{OUT_BASENAME}-{OUTTYPE}.gguf"

# 進捗UI
bar = FloatProgress(min=0, max=1, value=0, bar_style='info', layout={'width':'55%'})
label_head = HTML("<b>ダウンロード/変換:</b>")
label_tail = HTML("準備中…")
ui = HBox([label_head, bar, label_tail])
display(ui)

# 出力サイズの合計（バイト）。ログから検出できなければ None
total_bytes = None
last_size   = 0
stop_flag   = False
lines_q     = queue.Queue()

# 変換プロセスを静かに起動（警告を抑止）
env = dict(os.environ)
env["PYTHONWARNINGS"] = "ignore::FutureWarning,ignore::UserWarning"
cmd = [
    sys.executable, "-u",
    "-W", "ignore::FutureWarning", "-W", "ignore::UserWarning",
    conv_script, "--remote", MODEL_REPO, "--outtype", OUTTYPE, "--outfile", outfile_f16
]
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, bufsize=1, env=env)

def _reader(stream):
    for line in iter(stream.readline, ''):
        lines_q.put(line)
    stream.close()

t_out = threading.Thread(target=_reader, args=(p.stdout,), daemon=True)
t_err = threading.Thread(target=_reader, args=(p.stderr,), daemon=True)
t_out.start(); t_err.start()

# 単位→バイト換算
unit_tbl = {'K':1024, 'M':1024**2, 'G':1024**3, 'T':1024**4}
find_total = re.compile(r"total_size\s*=\s*([0-9]+(?:\.[0-9]+)?)\s*([KMGT])", re.I)

t0 = time.time()
try:
    while True:
        # ログをパースして total_size を取得（UIには出さない）
        while not lines_q.empty():
            line = lines_q.get_nowait()
            m = find_total.search(line)
            if m and total_bytes is None:
                val, unit = float(m.group(1)), m.group(2).upper()
                total_bytes = int(val * unit_tbl[unit])
                # 総サイズが分かったらプログレスを0%に初期化
                bar.value = 0
                label_tail.value = f"0.00 / {val:.2f}{unit}B (0%)"

        # 出力ファイルの伸びで%更新
        size_now = Path(outfile_f16).stat().st_size if Path(outfile_f16).exists() else 0
        if total_bytes:
            done = min(size_now / total_bytes, 1.0)
            bar.value = done
            done_gb = size_now / (1024**3)
            total_str = f"{total_bytes/(1024**3):.2f}GB"
            label_tail.value = f"{done_gb:.2f} / {total_str} ({done*100:.1f}%)"
        else:
            # まだ総サイズが不明：バーは0%のまま、状態だけ表示
            label_tail.value = "メタ情報取得中…（初回は時間がかかります）"

        if p.poll() is not None:
            # プロセス終了
            break
        time.sleep(0.2)

    # 終了処理
    rc = p.wait(timeout=5)
    # 最終更新
    if Path(outfile_f16).exists():
        size_now = Path(outfile_f16).stat().st_size
    if total_bytes:
        bar.value = 1.0 if rc == 0 else bar.value
        done_gb = size_now / (1024**3)
        total_str = f"{total_bytes/(1024**3):.2f}GB"
        label_tail.value = f"{done_gb:.2f} / {total_str} ({(size_now/total_bytes)*100:.1f}%)"
    else:
        # 総サイズ不明のまま終了した場合はサイズのみ表示
        label_tail.value = f"{size_now/(1024**3):.2f}GB 書き出し済み"

    if rc != 0:
        # 失敗時のみ最後の少量ログを出す（レイアウト崩れ対策）
        out, err = p.communicate(timeout=2)
        print("\n".join((out + "\n" + err).splitlines()[-30:]))
        raise SystemExit("convert_hf_to_gguf failed")

    bar.bar_style = 'success'
    elapsed = time.time() - t0
    label_head.value = "<b>完了:</b>"
    label_tail.value += f"  — {elapsed/60:.1f} 分"
finally:
    stop_flag = True

## 4. 量子化（例：Q5_K_M）

In [ ]:
# 4. 量子化（例：Q5_K_M） — RUNは素のテキスト表示、進捗バーは継続
import os, sys, time, shutil, subprocess, threading, queue
from pathlib import Path
from ipywidgets import FloatProgress, HBox, HTML
from IPython.display import display

BIN_DIR = "/opt/llama.cpp/build/bin"
llama_quantize = shutil.which("llama-quantize") or f"{BIN_DIR}/llama-quantize"
assert Path(llama_quantize).exists(), f"llama-quantize が見つかりません: {llama_quantize}"
assert 'outfile_f16' in globals() and Path(outfile_f16).exists(), "F16 GGUF が見つかりません。先に変換を完了させてください。"

outfile_quant = f"{OUT_DIR}/{OUT_BASENAME}-{QUANT}.gguf"

# 期待サイズ（概算）
bpw_guess = {"Q8_0":8.0,"Q6_K":6.3,"Q5_K_M":5.7,"Q5_K":5.5,"Q4_K_M":4.8,"Q4_1":4.5,"Q3_K_M":3.5,"Q2_K":2.6}
f16_bpw = 16.0
bpw = bpw_guess.get(QUANT.upper(), 5.7)
in_size = Path(outfile_f16).stat().st_size
expected_out = int(in_size * (bpw / f16_bpw))

# ── RUN 行（プレーン表示） ─────────────────────────
cmd = [llama_quantize, outfile_f16, outfile_quant, QUANT]
run_str = "RUN: " + " ".join(cmd)
print(run_str)

# ── 進捗バー UI ───────────────────────────────────
bar = FloatProgress(min=0, max=1, value=0, bar_style='info', layout={'width':'55%'})
label_head = HTML("<b>量子化:</b>")
label_tail = HTML("準備中…")
ui = HBox([label_head, bar, label_tail])
display(ui)

# ── 実行（静かめ） ─────────────────────────────────
env = dict(os.environ)
env["PYTHONWARNINGS"] = "ignore::FutureWarning,ignore::UserWarning"
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, bufsize=1, env=env)

lines_q = queue.Queue()
def _reader(stream):
    for line in iter(stream.readline, ''):
        lines_q.put(line)
    stream.close()

t_out = threading.Thread(target=_reader, args=(p.stdout,), daemon=True)
t_err = threading.Thread(target=_reader, args=(p.stderr,), daemon=True)
t_out.start(); t_err.start()

t0 = time.time()
try:
    while True:
        if Path(outfile_quant).exists():
            written = Path(outfile_quant).stat().st_size
            denom = max(expected_out, 1)
            frac = min(written / denom, 0.99)  # 完了前は 99% で止める
            bar.value = frac
            label_tail.value = f"{written/(1024**3):.2f} / {denom/(1024**3):.2f} GB ({frac*100:.1f}%)"
        else:
            label_tail.value = "出力ファイル作成待ち…"

        if p.poll() is not None:
            break
        time.sleep(0.25)

    rc = p.wait(timeout=5)

    if Path(outfile_quant).exists():
        written = Path(outfile_quant).stat().st_size
        bar.value = 1.0
        label_head.value = "<b>量子化: 完了</b>"
        label_tail.value = f"{written/(1024**3):.2f} GB  — {(time.time()-t0)/60:.1f} 分"
    else:
        label_tail.value = "出力が生成されませんでした"

    if rc != 0:
        out = []; err = []
        while not lines_q.empty():
            ln = lines_q.get_nowait()
            (err if "error" in ln.lower() else out).append(ln.rstrip())
        tail = "\n".join((out+err)[-30:])
        print("\n--- llama-quantize last logs ---\n" + tail)
        bar.bar_style = 'danger'
        raise SystemExit("quantize failed")

    bar.bar_style = 'success'
    print(f"OK -> {outfile_quant}")

finally:
    pass

## 5. 出力確認

In [ ]:
!ls -lh {OUT_DIR}/*.gguf || true

## 6. クイック動作確認（任意）

In [ ]:
# 6. クイック動作確認（ログ非表示・会話テキストのみ / 先頭復唱の自動除去）
import os, shlex, subprocess, shutil, re
from ipywidgets import HTML, VBox
from IPython.display import display

BIN_DIR = "/opt/llama.cpp/build/bin"
llama_cli = shutil.which("llama-cli") or f"{BIN_DIR}/llama-cli"

# ===== 調整ポイント =====
PROMPT      = "日本語で自己紹介してください。"
N_PREDICT   = 512
CONTEXT_LEN = 4096
BATCH       = 256
MODEL_PATH  = outfile_quant   # 量子化した GGUF を使う
# =======================

cmd = [
    llama_cli,
    "-m", MODEL_PATH,
    "--simple-io",                # 純テキストのみを stdout に
    "-p", PROMPT,
    "-n", str(N_PREDICT),
    "-c", str(CONTEXT_LEN),
    "-b", str(BATCH),
    "-ngl", "999",
    "--temp", "0.7", "--top-p", "0.95", "--seed", "0",
]
print("RUN:", " ".join(shlex.quote(x) for x in cmd))

# 先頭の「復唱」っぽいテキストを削るフィルタ
_prompt_pat = re.escape(PROMPT)
ECHO_RE = re.compile(
    r"^\s*(?:[「『\"]?\s*"+_prompt_pat+r"\s*[」』\"]?\s*[:：]?\s*){1,3}\s*",
    flags=re.DOTALL
)

def drop_leading_echo(text: str) -> str:
    # 先頭に連続する復唱（最大3回）をまとめて除去
    return ECHO_RE.sub("", text, count=1)

# ストリーミング描画
html = HTML("<div style='font-family:monospace; white-space:pre-wrap; line-height:1.6'></div>")
display(VBox([html]))

env = os.environ.copy()
env["LLAMA_LOG_LEVEL"]  = "1"   # errorのみ
env["LLAMA_LOG_COLORS"] = "0"

proc = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.DEVNULL,   # 赤ログを完全に消す
    text=True,
    bufsize=0,
    env=env,
)

buf = []
try:
    while True:
        chunk = proc.stdout.read(256)
        if not chunk:
            if proc.poll() is not None:
                break
            continue
        buf.append(chunk)
        text = "".join(buf)
        # 実行中は “一度だけ” フィルタを試す（復唱が来た瞬間を消す）
        text = drop_leading_echo(text)
        html.value = f"<div style='font-family:monospace; white-space:pre-wrap; line-height:1.6'>{text}</div>"
finally:
    proc.wait()
    text = drop_leading_echo("".join(buf))
    html.value = f"<div style='font-family:monospace; white-space:pre-wrap; line-height:1.6'>{text}</div>"

print(f"[exit] {proc.returncode}")